In [ ]:
import numpy as np
import pandas as pd

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from math import sqrt
import sys
import xgboost as xgb

In [ ]:
def optimize(random_state=SEED):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'eval_metric': 'rmse',
             'objective': 'reg:linear',
             'tree_method': 'exact',
             'silent' : 1,
             'seed': random_state
             }

    best = fmin(score, space, algo=tpe.suggest, max_evals=250)

    print best


In [1]:
def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(train_features, label=y_train)
    dvalid = xgb.DMatrix(valid_features, label=y_valid)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = sqrt(mean_squared_error(y_valid, predictions))
    # TODO: Add the importance for the selected features
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = score
    return {'loss': loss, 'status': STATUS_OK}

In [ ]:
cols = [c for c in X_train.columns if c not in ['date_block_num', 'item_cnt_day',]]

test = pd.read_csv('test.csv')
X_train = pd.read_csv('X_train.csv')
X_cv = pd.read_csv('X_cv.csv')
X_test = pd.read_csv('X_test.csv')

train_feature = X_train[cols]
y_train X_train['item_cnt_day']
valid_features = X_cv[cols]
y_valid = X_cv['item_cnt_day']

best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)
